<a href="https://colab.research.google.com/github/mrtlckn/IntroToPytorch/blob/main/102_DiffusionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
class Diffusion:
  def __init__(self, noise_steps= 1000, beta_start = 1e-4, beta_end = 0.02, img_size = 64):
    self.noise_steps = noise_steps
    self.beta_start = beta_start
    self.beta_end = beta_end
    self.img_size = img_size

    self.beta = self.prepate_noise_schedule
    self.alpha = 1. - self.beta
    self.alpha_hat = torch.cumprod(self.alpha, dim = 0)

  def prepare_noise_schedule(self):
    return torch.linspace(self.beta_start, self.beta_end, self.noise_steps)